# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [49]:
import pandas as pd
import numpy as np
import requests
import json
import xlsxwriter
import math
from scipy import stats
from secrets import IEX_CLOUD_API_TOKEN

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [50]:
stocks=pd.read_csv('sp_500_stocks.csv')
stocks.head()

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV


## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [71]:
url=f"https://sandbox.iexapis.com/stable/stock/aapl/stats?token={IEX_CLOUD_API_TOKEN}"
r=requests.get(url).json()
url= f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string[0]}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
r=requests.get(url).json()

r

{'Y': {'advanced-stats': {'beta': 0.4567051077882094,
   'totalCash': None,
   'currentDebt': 21596272456,
   'revenue': 12268491690,
   'grossProfit': 12362384268,
   'totalRevenue': 12597462096,
   'EBITDA': 1410894358,
   'revenuePerShare': 937.94,
   'revenuePerEmployee': 2787875.05,
   'debtToEquity': 3.888678806011948,
   'profitMargin': 0.08081395809431784,
   'enterpriseValue': 11468574098,
   'enterpriseValueToRevenue': 0.9531,
   'priceToSales': 0.927,
   'priceToBook': 1.3,
   'forwardPERatio': 11.784507404999133,
   'pegRatio': 0.31103379343524856,
   'peHigh': 13.168219035344233,
   'peLow': 10.44935655389347,
   'week52highDate': '2022-03-24',
   'week52lowDate': '2022-02-23',
   'putCallRatio': None,
   'companyName': 'Alleghany Corp.',
   'marketcap': 11694650109,
   'week52high': 883.48,
   'week52low': 614.3,
   'week52highSplitAdjustOnly': 893.86,
   'week52highDateSplitAdjustOnly': '2022-03-11',
   'week52lowSplitAdjustOnly': 585.7,
   'week52lowDateSplitAdjustOnly'

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [52]:
per=r['peRatio']
per


21.72750570323323

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [53]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [56]:
final_df=pd.DataFrame(columns=my_columns)
for symbol_string in symbol_strings:
    
    url= f'https://sandbox.iexapis.com/stable/stock/market/batch/?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    r=requests.get(url).json()
    
    
    for symbol in r.keys():
        df=pd.DataFrame(np.array([symbol,
                                  r[symbol]['quote']['latestPrice'],
                                  r[symbol]['quote']['peRatio'],
                                'N/A'
                                 ]).reshape(1,-1),columns=my_columns)
        
        final_df=pd.concat([final_df,df],ignore_index=True)
        
        
final_df.head()

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,113.84,28.18,N/A
1,AAL,13.45,-3.68,N/A
2,AAP,175.83,18.76,N/A
3,AAPL,134.22,21.47,N/A
4,ABBV,142.68,19.84,N/A


In [69]:
r

{'YUM': {'quote': {'avgTotalVolume': 1507646,
   'calculationPrice': 'close',
   'change': 0.37,
   'changePercent': 0.0034,
   'close': 113.8,
   'closeSource': 'lifaiocf',
   'closeTime': 1678603211845,
   'companyName': 'Yum Brands Inc.',
   'currency': 'USD',
   'delayedPrice': 113.52,
   'delayedPriceTime': 1724792823038,
   'extendedChange': 0,
   'extendedChangePercent': 0,
   'extendedPrice': 110.8,
   'extendedPriceTime': 1662678540332,
   'high': 114.83,
   'highSource': 'riictd ne p1dmuee 5laye',
   'highTime': 1670491026168,
   'iexAskPrice': None,
   'iexAskSize': None,
   'iexBidPrice': None,
   'iexBidSize': None,
   'iexClose': 114.73,
   'iexCloseTime': 1702138953305,
   'iexLastUpdated': None,
   'iexMarketPercent': None,
   'iexOpen': 114.411,
   'iexOpenTime': 1667220371442,
   'iexRealtimePrice': None,
   'iexRealtimeSize': None,
   'iexVolume': None,
   'lastTradeTime': 1672759094605,
   'latestPrice': 112.2,
   'latestSource': 'Close',
   'latestTime': 'June 17, 

## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [57]:
final_df['Price-to-Earnings Ratio']=pd.to_numeric(final_df['Price-to-Earnings Ratio'])
final_df=final_df[final_df['Price-to-Earnings Ratio']>0]
final_df.sort_values('Price-to-Earnings Ratio',inplace=True,ascending=True)


final_value_df=final_df[:50]

## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [58]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")
        
final_value_df

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
338,NRG,38.03,2.28,N/A
151,EBAY,42.45,2.40,N/A
36,AMZN,111.38,2.56,N/A
173,F,11.53,3.98,N/A
342,NUE,116.36,4.20,N/A
424,SYF,30.92,4.24,N/A
106,COF,109.06,4.30,N/A
132,DHI,61.72,4.47,N/A
368,PHM,37.62,4.53,N/A
18,AIG,52.58,4.60,N/A


Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [59]:
final_value_df.reset_index(drop=True,inplace=True)

You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [63]:
portfolio_input()
position_size=float(portfolio_size)/50
position_size

Enter the value of your portfolio:10000000


200000.0

In [66]:
for row in final_value_df.index:
    final_value_df.loc[row,'Number of Shares to Buy']=math.floor(position_size/float(final_value_df.loc[row,'Price']))
    

C:\anaconda\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [67]:
final_value_df

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,38.03,2.28,5259
1,EBAY,42.45,2.40,4711
2,AMZN,111.38,2.56,1795
3,F,11.53,3.98,17346
4,NUE,116.36,4.20,1718
5,SYF,30.92,4.24,6468
6,COF,109.06,4.30,1833
7,DHI,61.72,4.47,3240
8,PHM,37.62,4.53,5316
9,AIG,52.58,4.60,3803


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [88]:
my_columns=[ 'Ticker', 
            'Price',  
            'Number of Shares to Buy',
            'Price-to-Earnings Ratio',
            'PE Percentile',
           'Price-to-Book Ratio',
            'PB Percentile',
            'EV/EBITDA',
            'EV/EBITDA Percentile',
            'EV/GP',
            'EV/GP Percentile',
            'RV Score'
           ]


final_df=pd.DataFrame(columns=my_columns)
for symbol_string in symbol_strings:
    
    url= f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    r=requests.get(url).json()
    
    
    for symbol in r.keys():
        if r[symbol]['advanced-stats']['enterpriseValue']!=None and r[symbol]['advanced-stats']['EBITDA']!=None:
            
            df=pd.DataFrame(np.array([symbol,
                                      r[symbol]['quote']['latestPrice'],
                                      'N/A',

                                      r[symbol]['quote']['peRatio'],
                                      'N/A',

                                      r[symbol]['advanced-stats']['priceToBook'],
                                      'N/A',
                                      
                                      (r[symbol]['advanced-stats']['enterpriseValue']/r[symbol]['advanced-stats']['EBITDA']),
                                      'N/A',
                                      

                                      (r[symbol]['advanced-stats']['enterpriseValue']/r[symbol]['advanced-stats']['grossProfit']),
                                      'N/A',
                                      'N/A'

                                     ]).reshape(1,-1),columns=my_columns,dtype=(str,float))
            
        else:
            df=pd.DataFrame(np.array([symbol,
                                      r[symbol]['quote']['latestPrice'],
                                      'N/A',

                                      r[symbol]['quote']['peRatio'],
                                      'N/A',

                                      r[symbol]['advanced-stats']['priceToBook'],
                                      'N/A',
                                      np.NAN,
    #                                   (r[symbol]['advanced-stats']['enterpriseValue']/r[symbol]['advanced-stats']['EBITDA']),
                                      'N/A',
                                      np.NAN,

    #                                    (r[symbol]['advanced-stats']['enterpriseValue']/r[symbol]['advanced-stats']['grossProfit']),
                                      'N/A',
                                      'N/A'

                                     ]).reshape(1,-1),columns=my_columns,dtype=(str,float))
            
        
        final_df=pd.concat([final_df,df],ignore_index=True)
        
        
final_df.head()

# priceToSales
# priceToBook

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,113.35,N/A,28.18,N/A,6.76,N/A,19.602729163250494,N/A,9.797078385224957,N/A,N/A
1,AAL,13.29,N/A,-3.58,N/A,-0.9857,N/A,-21.87281502380507,N/A,0.9984990282412457,N/A,N/A
2,AAP,170.65,N/A,19.32,N/A,3.6,N/A,10.674726802350273,N/A,2.2476663509945616,N/A,N/A
3,AAPL,137.1,N/A,22.2,N/A,31.97,N/A,16.73853312712821,N/A,13.200872747875128,N/A,N/A
4,ABBV,140.72,N/A,20.4,N/A,15.1,N/A,10.446141425791625,N/A,7.811027757471554,N/A,N/A


Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,100.010,N/A,46.95,N/A,6.596140,N/A,26.372976,N/A,11.311629,N/A,N/A
1,AAL,13.360,N/A,-1.71,N/A,-60.417952,N/A,5.952664,N/A,3.098483,N/A,N/A
2,AAP,161.440,N/A,29,N/A,3.123759,N/A,15.086368,N/A,3.208667,N/A,N/A
3,AAPL,466.070,N/A,34.69,N/A,22.373999,N/A,25.708773,N/A,19.428993,N/A,N/A
4,ABBV,96.680,N/A,21,N/A,-21.463532,N/A,12.272585,N/A,7.672427,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,94.320,N/A,28,N/A,-3.659682,N/A,18.841249,N/A,13.891510,N/A,N/A
501,ZBH,143.470,N/A,718.1,N/A,2.390128,N/A,17.170711,N/A,7.478460,N/A,N/A
502,ZBRA,288.222,N/A,31.86,N/A,8.600669,N/A,19.480804,N/A,8.449885,N/A,N/A
503,ZION,35.770,N/A,13.24,N/A,0.766237,N/A,NaN,N/A,NaN,N/A,N/A


{'week52change': 0.290414, 'week52high': 169.51, 'week52low': 90.41, 'marketcap': 75861250162, 'employees': 10724, 'day200MovingAvg': 134.17, 'day50MovingAvg': 148.64, 'float': 479955020, 'avg10Volume': 1641586.3, 'avg30Volume': 1479579.2, 'ttmEPS': 3.499, 'ttmDividendRate': 0.77, 'companyName': 'Zoetis, Inc.', 'sharesOutstanding': 494883276, 'maxChangePercent': 3.3525, 'year5ChangePercent': 1.69, 'year2ChangePercent': 0.608, 'year1ChangePercent': 0.281583, 'ytdChangePercent': 0.189876, 'month6ChangePercent': 0.10241, 'month3ChangePercent': 0.297048, 'month1ChangePercent': 0.159224, 'day30ChangePercent': 0.165634, 'day5ChangePercent': -0.012817, 'nextDividendDate': None, 'dividendYield': 0.00489706569361443, 'nextEarningsDate': '2020-10-13', 'exDividendDate': '2020-07-01', 'peRatio': 49.1, 'beta': 0.9584403132945124, 'totalCash': 1974321215, 'currentDebt': 549171552, 'revenue': 6518431250, 'grossProfit': 4133660573, 'totalRevenue': 6300683466, 'EBITDA': 2502772553, 'revenuePerShare': 1

## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [89]:
final_df[final_df.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
28,ALXN,184.3,N/A,59.78,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
40,AON,252.61,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,271.78,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
88,CERN,95.74,N/A,48.16,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11,N/A,10.02,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
135,DISCK,24.58,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
164,ETFC,49.89,N/A,14.94,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
185,FLIR,60.04,N/A,32.38,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
189,FOX,31.06,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
192,FRT,98.81,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [95]:
for column in ['Price-to-Earnings Ratio','Price-to-Book Ratio','EV/EBITDA','EV/GP']:
    final_df[column].fillna(pd.to_numeric(final_df[column]).mean(),inplace=True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [96]:
final_df[final_df.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [108]:
percentiles={'PE Percentile':'Price-to-Earnings Ratio',
           'PB Percentile':'Price-to-Book Ratio',
            'EV/EBITDA Percentile':'EV/EBITDA',
            'EV/GP Percentile':'EV/GP'
           }

for col in [  
            'Price',  
            'Price-to-Earnings Ratio',
            'PE Percentile',
           'Price-to-Book Ratio',
            'PB Percentile',
            'EV/EBITDA',
            'EV/EBITDA Percentile',
            'EV/GP',
            'EV/GP Percentile',
            
           ]:
    final_df[col]=pd.to_numeric(final_df[col])


for row in final_df.index:
    for percentile in percentiles.keys():
        
        final_df.loc[row,percentile]=stats.percentileofscore(final_df[percentiles[percentile]],final_df.loc[row,percentiles[percentile]])/100

## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [138]:
final_df['RV Score']=0
from statistics import mean
for row in final_df.index:
    final_df.loc[row,'RV Score']=mean([final_df.loc[row,'PE Percentile'],final_df.loc[row,'PB Percentile'],final_df.loc[row,'EV/EBITDA Percentile'],final_df.loc[row,'EV/GP Percentile']])

## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [139]:
final_df.sort_values('RV Score', inplace=True)
final_df_value=final_df[:50]
final_df_value.reset_index(inplace=True,drop=True)
final_df_value

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AAL,13.29,15048908,-3.58,0.033932,-0.9857,0.059880,-21.872815,0.007984,0.998499,0.039920,0.035429
1,AIG,52.16,3834355,4.50,0.063872,0.7403,0.075848,2.738997,0.027944,0.778975,0.021956,0.047405
2,COF,107.78,1855631,4.20,0.057884,0.7632,0.077844,2.878239,0.031936,1.283373,0.067864,0.058882
3,UNM,33.85,5908419,7.40,0.117764,0.6165,0.071856,3.796399,0.051896,0.566506,0.007984,0.062375
4,C,47.39,4220299,5.53,0.085828,0.5291,0.063872,3.511824,0.045908,1.121050,0.061876,0.064371
5,SYF,30.14,6635700,4.09,0.053892,1.1800,0.143713,2.588305,0.025948,0.964564,0.033932,0.064371
6,GM,31.98,6253908,5.42,0.079840,0.7688,0.079840,1.775798,0.019960,1.427121,0.077844,0.064371
7,LNC,48.89,4090816,6.92,0.109780,0.5600,0.065868,4.849270,0.095808,0.420785,0.001996,0.068363
8,APA,40.85,4895960,6.00,0.095808,-770.5700,0.001996,3.829369,0.053892,2.019149,0.123752,0.068862
9,PVH,63.35,3157063,4.54,0.065868,0.8005,0.083832,4.767544,0.091816,1.066147,0.053892,0.073852


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [141]:
portfolio_input()
val=float(portfolio_size)/50

Enter the value of your portfolio:10000000000


In [150]:
for row in final_df_value.index:
    final_df_value.loc[row,'Number of Shares to Buy']=math.floor(val/float(final_df_value.loc[row,'Price']))
final_df_value.head()    

C:\anaconda\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AAL,13.29,15048908,-3.58,0.033932,-0.9857,0.059880,-21.872815,0.007984,0.998499,0.039920,0.035429
1,AIG,52.16,3834355,4.50,0.063872,0.7403,0.075848,2.738997,0.027944,0.778975,0.021956,0.047405
2,COF,107.78,1855631,4.20,0.057884,0.7632,0.077844,2.878239,0.031936,1.283373,0.067864,0.058882
3,UNM,33.85,5908419,7.40,0.117764,0.6165,0.071856,3.796399,0.051896,0.566506,0.007984,0.062375
4,C,47.39,4220299,5.53,0.085828,0.5291,0.063872,3.511824,0.045908,1.121050,0.061876,0.064371


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [147]:
writer=pd.ExcelWriter('value_investing_strategy.xlsx',engine='xlsxwriter')
final_df_value.to_excel(writer,sheet_name="Value Investing Strategy",index=False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [148]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
)
float_template = writer.book.add_format(
        {
            'num_format':'0.0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )    
    
  
    
    

In [149]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile', percent_template],
                    'H': ['EV/EBITDA', float_template],
                    'I': ['EV/EBITDA Percentile', percent_template],
                    'J': ['EV/GP', float_template],
                    'K': ['EV/GP Percentile', percent_template],
                    'L': ['RV Score', percent_template]
                    }

for col in column_formats.keys():
    writer.sheets['Value Investing Strategy'].set_column(f'{col}:{col}',22,column_formats[col][1])
    writer.sheets['Value Investing Strategy'].write(f'{col}1',column_formats[col][0],column_formats[col][1])
    
    
writer.save()